In [10]:
import os
import re
import pandas as pd
import spacy
import en_core_web_sm

In [5]:
def extract_chunks(input_text):
    doc = nlp(input_text)
    spans = list(doc.ents)  #+ list(doc.noun_chunks)
    out_list = []
    for span in spans:
        span.merge()
        out_list.append(span)
    return out_list


In [6]:
def extract_name_pair(name, input_text):
    out_list = []
    if not pd.isnull(input_text):
        s_out = extract_chunks(input_text)    
        doc = nlp(input_text)
        spans = list(doc.ents)  #+ list(doc.noun_chunks)

        for item in spans:
            item = re.sub(pattern='[^a-zA-Z0-9 -]',repl='',string=item.string)            
            if (re.search(item,name,flags=re.IGNORECASE)):
                out_list.append(item)

    return out_list
#     else:
#         return None

In [7]:
def basic_wc(input_text):
    
    if (input_text and not pd.isnull(input_text) and not None):
        decoded_txt=input_text.decode('utf-8')
        tokens = word_tokenize(decoded_txt)
        tokens = [tok for tok in tokens if (not re.search('^[^a-zA-Z]*$',tok))]
        return len(tokens)
    else:
        return 0    

In [12]:
nlp = en_core_web_sm.load()
out_analyst= pd.DataFrame.from_csv('analyst_out_foringestion.csv',encoding='utf-8')
out_analyst['analyst_exec_raw'] = out_analyst.apply(lambda x: extract_name_pair(x['exec_name'],x['analyst_qanda']), axis=1)
out_analyst['exec_name_count'] = out_analyst.apply(lambda x: \
                                         len(x['analyst_exec_raw']), axis=1)
out_analyst[['file','quarter_year','executive_firm','ticker','analyst',\
'analyst_name','analyst_firm','executive','analyst_exec_raw','exec_name_count']]\
.to_csv('analyst_out_withcount.csv',sep=',', encoding = 'utf-8')
# out_executives.select({[}'file','quarter_yer'])